In [ ]:
import pandas as pd

# Example DataFrame
df = pd.DataFrame({
    'my_column': ['A;B', 'B;C', 'A', 'C;D', 'B']
})

# Split the column by ';' into lists
df['my_column_split'] = df['my_column'].str.split(';')

# Explode the lists into rows
exploded = df.explode('my_column_split')

# Create binary columns for each unique value
dummies = pd.get_dummies(exploded['my_column_split'])

# Group back to original rows by summing
binary_df = dummies.groupby(exploded.index).sum()

# Concatenate with original DataFrame if needed
result = pd.concat([df, binary_df], axis=1)

result